In [2]:
#required imports cell, run first
import torch
import numpy as np
import pandas as pd
import torch_geometric.transforms as T
import torch.nn.functional as F
from torch_geometric.datasets import OGB_MAG
from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.loader import NeighborLoader
from torch_geometric.data.hetero_data import HeteroData

# Heterogeneous SBM Creation

In [3]:
#based on the below article in tandem with drake's csbm point cloud feature data build
#https://hal.archives-ouvertes.fr/hal-03442935/document

In [4]:
#independent degree HeSBM (first proposed model in paper)
def gen_ind_deg_HeSBM(N, M, K):
    """
    Generates an independent-degree heterogeneous stochastic blockmodel
    Arguments:
        N: number of nodes
        M: number of types
        K: number of communities
    """
    A = np.zeros((N,N)) #adjacency matrix
    pi = np.zeros((K,K)) #group interrelationship matrix
    theta = np.zeros((N,M,M)) #theta^(ml)_i is supposed to control the expected degree of vertex i in layer ml.
    
    z = np.random.randint(0, K, N) #community assignments, indexed by node #
    node_type = np.random.randint(0, M, N) #node type assignments ""
    
    #generate theta, enforcing conditions (sums over community members of given theta is 1)
    for i in range(N):
        for m in range(M):
            for l in range(m, M):
                theta[i, m, l] = theta[i, l, m] = (1 - np.random.power(3.5)) #power law for degree influence
    
    #now enforce theta community sum property by dividing by totals
    for m in range(M):
        for l in range(M):
            for k in range(K):
                idx_k = np.where(z==k)[0]
                total = 0
                
                for node in idx_k:
                    total += theta[node,m,l]
                    
                for node in idx_k:
                    theta[node,m,l] = theta[node,m,l] / total
                
    #generate pi (arbitrary)
    for q in range(K):
        for s in range(q, K):
            if q == s:
                pi[q, s] = 0.8
            else:
                pi[q, s] = pi[s, q] = (0.2 + np.random.random() * 0.1 * (-1 if np.random.random() < 0.5 else 1))
        
    
    max_p = 0
    #now compute adjacency
    for i in range(N):
        for j in range(i+1, N):
            m = node_type[i]#node type of i
            l = node_type[j]#node type of j
            q = z[i] #community of i
            s = z[j] #community of j
            
            p_ij  = theta[i,m,l] * theta[j,m,l] * pi[q,s] * N
            
            if p_ij > max_p: max_p = p_ij
            
            A[i,j] = A[j,i] = np.random.poisson(p_ij)
            
    return A, node_type, z

# HeteroData Compatible HeteroSBM

In [5]:
#IMPORTANT FOR CSBMs
def generate_perms_rand(num_classes,num_feat):
    vecs = []
    for i in range(num_classes):
        orthogonal = False
        while(not orthogonal):
            rand_vec = np.random.uniform(-10,10,num_feat)
            rand_vec = rand_vec/np.linalg.norm(rand_vec)
            orthogonal = True
            for i in range(len(vecs)):
                angle = np.arccos(rand_vec@vecs[i])
                if(angle < np.pi/2 - .1 or angle > np.pi/2 + .1):
                    orthogonal = False
        vecs.append(rand_vec)
        #print(len(vecs))
    return np.array(vecs)

def gen_feature_data(d, lamb, mu, num_features, num_nodes,num_classes, communities):
    c_in = d+lamb*np.sqrt(d) # c_in/c_out as described in the equations
    c_out = d-lamb*np.sqrt(d) 
    p_in = c_in/num_nodes # compiles these to pass into the SSBM
    p_out = c_out/num_nodes
    
    u = np.random.normal(0,1/num_features,(num_features)) # obtains the random normal vector u
    Z = np.random.normal(0,.2,(num_nodes,num_features)) # obtains the random noise vector i presume
    v = communities # puts the groups into a format for the equations
    
    perms = generate_perms_rand(num_classes,num_features)
    
    b = np.zeros((num_nodes,num_features))
    for i in range(num_nodes):
        b[i] = np.sqrt(mu/num_nodes)*(np.diag(perms[v[i]])@u) + Z[i]/np.sqrt(num_features)
    return b


In [6]:
from torch_geometric.data.hetero_data import HeteroData

def gen_heterodata_HSBM(
    N, M, K, 
    schema=None, 
    pi=None, 
    pi_intra=0.8, 
    pi_inter=0.3, 
    type_splits=None,
    n_feats=10,
    undirected=True
    ):
    
    data = HeteroData()
    node_type = [f'node{i}' for i in range(0, M)]
    
    if schema==None:
        #default schema is all possible layers
        schema = np.ones((M,M), dtype=np.int32)
    
    if pi == None:
        #default to simple group connectivity matrix
        diag = pi_intra * np.identity(K)
        nondiag = pi_inter * (np.ones((K,K)) - np.identity(K))
        pi = diag + nondiag
        
    if type_splits == None:
        #default to ~almost even split among M types
        block_size = int(N/M)
        type_splits = [block_size for i in range(0,M-1)]
        type_splits.append(N - block_size*(M-1))
    
    #if type(n_feats) == int:
        #even # feats for each node type
        #num = n_feats
        #n_feats = [num for i in range(0, M)]
        
    #assign classes uniformly by node type
    class_assignments=[]
    for m in range(0,M):
        assignment = np.random.randint(0, K, type_splits[m])
        data[node_type[m]].y = torch.tensor(assignment)
        class_assignments.append(assignment)
        
    
    #TODO:add feature generation
    for m in range(0, M):
        features = gen_feature_data(1, 0.5, 250, n_feats, type_splits[m], K, class_assignments[m])
        data[node_type[m]].x = torch.tensor(features, dtype=torch.float32)
    
    #graph generation, loop through layers
    for m in range(0,M):
        for l in range(m, M):
            if schema[m,l] != 0:
                edge_type = f"{m}_to_{l}"
                edge_type_rev = f"{l}_to_{m}"
                layer_key = (node_type[m], edge_type, node_type[l])
                layer_key_rev = (node_type[l], edge_type_rev, node_type[m])
                
                data[layer_key].edge_index = torch.tensor([], dtype=torch.long)
                
                for i in range(0, type_splits[m]):
                    for j in range(0, type_splits[l]):
                        theta_iml = 0.25
                        theta_jml = 0.25
                        p_ij = theta_iml * theta_jml * pi[m,l]
                        
                        if np.random.poisson(p_ij) == 1:
                            x = data[layer_key].edge_index
                            ij = torch.tensor([i,j])
                            data[layer_key].edge_index = torch.cat((x, ij), 0)
                
                data[layer_key].edge_index = torch.reshape(data[layer_key].edge_index, (-1, 2)) 
                data[layer_key].edge_index.transpose_(0,1)
                
                if undirected and m != l:
                    data[layer_key_rev].edge_index = data[layer_key].edge_index[[1,0],:]                    
    return data

data = gen_heterodata_HSBM(1000,3,3)
data['node0', '0_to_0', 'node0']

{'edge_index': tensor([[  0,   0,   0,  ..., 332, 332, 332],
        [  0,  57,  72,  ..., 254, 288, 326]])}